## Week 3 Assignment

In [761]:
import pandas as pd
from pandas import DataFrame
from geopy.geocoders import Nominatim
import folium
from folium.plugins import MarkerCluster
import requests
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

In [762]:
df=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [763]:
df=df[0]

In [764]:
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## Ignoring Rows which have Borough as 'Not assigned'

In [765]:
df=df[df['Borough']!='Not assigned']

In [766]:
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## Checking if any Not assigned values exist in Neighborhood

In [767]:
df[df['Neighbourhood']=='Not assigned']

,Postal Code,Borough,Neighbourhood


In [768]:
df.shape

(103, 3)

In [769]:
## Part2 of the Week 3 Assignment

In [770]:
!wget -O geo.csv http://cocl.us/Geospatial_data

--2020-07-29 19:23:57--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 159.8.69.21, 159.8.69.24, 159.8.72.228
Connecting to cocl.us (cocl.us)|159.8.69.21|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2020-07-29 19:23:57--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|159.8.69.21|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-07-29 19:23:58--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 185.235.236.197
Connecting to ibm.box.com (ibm.box.com)|185.235.236.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-07-29 19:23:58--  https://ibm.box.com/public/static/

In [771]:
geodata=pd.read_csv('geo.csv')

In [772]:
geodata.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [773]:
dfcomp=df.merge(geodata,left_on='Postal Code',right_on='Postal Code')

In [774]:
dfcomp.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [775]:
dfcomp['Borough'].unique()

array(['North York', 'Downtown Toronto', 'Etobicoke', 'Scarborough',
       'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

In [776]:
dff=dfcomp[dfcomp['Borough'].str.match(r'(?=.*\bToronto\b)')==True]

In [777]:
dff.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [778]:
dff.shape

(39, 5)

In [779]:
dff.sort_values(by=['Borough'],inplace=True)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [780]:
dff.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
62,M5N,Central Toronto,Roselawn,43.711695,-79.416936
67,M4P,Central Toronto,Davisville North,43.712751,-79.390197
68,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307
73,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678
74,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678


In [781]:
dff.drop(columns=['Postal Code'],inplace=True)

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [782]:
dff.head()

,Borough,Neighbourhood,Latitude,Longitude
62,Central Toronto,Roselawn,43.711695,-79.416936
67,Central Toronto,Davisville North,43.712751,-79.390197
68,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307
73,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678
74,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678


## Deciding to explore Downtown Toronto for clustering

In [783]:
dffdt=dff[dff['Borough']=='Downtown Toronto']

In [784]:
dffdt.head()

,Borough,Neighbourhood,Latitude,Longitude
84,Downtown Toronto,"Kensington Market, Chinatown, Grange Park",43.653206,-79.400049
99,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
80,Downtown Toronto,"University of Toronto, Harbord",43.662696,-79.400049
87,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420
91,Downtown Toronto,Rosedale,43.679563,-79.377529


In [785]:
dt=Nominatim(user_agent='ca')

In [786]:
location = dt.geocode('Toronto,CA')
lat = location.latitude
long = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6534817, -79.3839347.


In [787]:
mapca=folium.Map(location=[lat,long],zoom_start=13)
for label,lat,long in zip(dffdt['Neighbourhood'],dffdt['Latitude'],dffdt['Longitude']):
    folium.CircleMarker(location=[lat,long],popup=label,fill_color='Red').add_to(mapca)
mapca

In [788]:
ClientID='IHLKQKVCF4VB42G1VXQE23KSOSEZUYKVJZDRZVLDZ01WEUGJ'
ClientSecret='KTZ24UAVDVUIEKFRWJW13LHWYCVKOJHGN4IN1DWI3FOXX4DS'
ver='20200101'


In [789]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    l=100
    for name, lat, lng in zip(names, latitudes, longitudes):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            ClientID, 
            ClientSecret, 
            ver, 
            lat, 
            lng, 
            radius, 
            l)
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Lat', 
                  'Neighbourhood Long', 
                  'Venue', 
                  'Venue Lat', 
                  'Venue Long', 
                  'Venue Category']
    return(nearby_venues)

In [790]:
dt_venues = getNearbyVenues(names=dffdt['Neighbourhood'],
                                   latitudes=dffdt['Latitude'],
                                   longitudes=dffdt['Longitude']
                                  )

In [791]:
dt_venues.head()

,Neighbourhood,Neighbourhood Lat,Neighbourhood Long,Venue,Venue Lat,Venue Long,Venue Category
0,"Kensington Market, Chinatown, Grange Park",43.653206,-79.400049,Seven Lives - Tacos y Mariscos,43.654418,-79.400545,Mexican Restaurant
1,"Kensington Market, Chinatown, Grange Park",43.653206,-79.400049,Essence of Life Organics,43.654111,-79.400431,Organic Grocery
2,"Kensington Market, Chinatown, Grange Park",43.653206,-79.400049,Kid Icarus,43.653933,-79.401719,Arts & Crafts Store
3,"Kensington Market, Chinatown, Grange Park",43.653206,-79.400049,Banh Mi Nguyen Huong,43.653628,-79.398376,Vietnamese Restaurant
4,"Kensington Market, Chinatown, Grange Park",43.653206,-79.400049,The Moonbean Cafe,43.654147,-79.400182,Café


In [792]:
dfff=pd.get_dummies(dt_venues['Venue Category'],prefix="",prefix_sep="")

In [793]:
dfff.head()

,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [794]:
dt_venues.drop(columns='Venue Category',inplace=True)

In [795]:
dt_venues.head()

,Neighbourhood,Neighbourhood Lat,Neighbourhood Long,Venue,Venue Lat,Venue Long
0,"Kensington Market, Chinatown, Grange Park",43.653206,-79.400049,Seven Lives - Tacos y Mariscos,43.654418,-79.400545
1,"Kensington Market, Chinatown, Grange Park",43.653206,-79.400049,Essence of Life Organics,43.654111,-79.400431
2,"Kensington Market, Chinatown, Grange Park",43.653206,-79.400049,Kid Icarus,43.653933,-79.401719
3,"Kensington Market, Chinatown, Grange Park",43.653206,-79.400049,Banh Mi Nguyen Huong,43.653628,-79.398376
4,"Kensington Market, Chinatown, Grange Park",43.653206,-79.400049,The Moonbean Cafe,43.654147,-79.400182


In [796]:
result = pd.concat([dt_venues, dfff], axis=1, sort=False)

In [797]:
result.head()

,Neighbourhood,Neighbourhood Lat,Neighbourhood Long,Venue,Venue Lat,Venue Long,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Kensington Market, Chinatown, Grange Park",43.653206,-79.400049,Seven Lives - Tacos y Mariscos,43.654418,-79.400545,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Kensington Market, Chinatown, Grange Park",43.653206,-79.400049,Essence of Life Organics,43.654111,-79.400431,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Kensington Market, Chinatown, Grange Park",43.653206,-79.400049,Kid Icarus,43.653933,-79.401719,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Kensington Market, Chinatown, Grange Park",43.653206,-79.400049,Banh Mi Nguyen Huong,43.653628,-79.398376,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,"Kensington Market, Chinatown, Grange Park",43.653206,-79.400049,The Moonbean Cafe,43.654147,-79.400182,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [798]:
resultg=result.groupby('Neighbourhood').mean().reset_index()

In [799]:
resultg.head(10)

,Neighbourhood,Neighbourhood Lat,Neighbourhood Long,Venue Lat,Venue Long,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,43.644771,-79.373306,43.647199,-79.373573,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.0,0.00,0.017241,0.0,0.000000,0.000000,0.0,0.0,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420,43.630901,-79.395626,0.000000,0.071429,0.071429,0.142857,0.142857,...,0.00,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000
2,Central Bay Street,43.657952,-79.387383,43.658391,-79.386006,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.0,0.00,0.016129,0.0,0.000000,0.016129,0.0,0.0,0.016129
3,Christie,43.669542,-79.422564,43.670525,-79.423731,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000
4,Church and Wellesley,43.665860,-79.383160,43.666195,-79.382816,0.013158,0.000000,0.000000,0.000000,0.000000,...,0.00,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.026316
5,"Commerce Court, Victoria Hotel",43.648199,-79.379817,43.648847,-79.379601,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.0,0.00,0.020000,0.0,0.000000,0.010000,0.0,0.0,0.000000
6,"First Canadian Place, Underground city",43.648429,-79.382280,43.648801,-79.382441,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.0,0.01,0.010000,0.0,0.000000,0.010000,0.0,0.0,0.000000
7,"Garden District, Ryerson",43.657162,-79.378937,43.656159,-79.380621,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.01,0.0,0.00,0.000000,0.0,0.010000,0.010000,0.0,0.0,0.000000
8,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,43.641369,-79.382588,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.0,0.01,0.010000,0.0,0.000000,0.010000,0.0,0.0,0.000000
9,"Kensington Market, Chinatown, Grange Park",43.653206,-79.400049,43.654067,-79.401664,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.0,0.00,0.061538,0.0,0.046154,0.015385,0.0,0.0,0.000000


In [800]:
res=resultg.drop(columns=['Neighbourhood Lat','Neighbourhood Long','Venue Lat','Venue Long'])

In [801]:
res.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017241,0.0,0.0,...,0.0,0.0,0.0,0.017241,0.0,0.0,0.000000,0.0,0.0,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.071429,0.071429,0.142857,0.142857,0.142857,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
2,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.016129,0.0,0.0,0.016129,0.0,0.0,0.016129
3,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
4,Church and Wellesley,0.013158,0.000000,0.000000,0.000000,0.000000,0.000000,0.013158,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.026316


In [802]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [803]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = res['Neighbourhood']
for ind in np.arange(res.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(res.iloc[ind, :], num_top_venues)
neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Bakery,Cocktail Bar,Café,Farmers Market,Beer Bar,Seafood Restaurant,Restaurant,Cheese Shop,Basketball Stadium
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Boat or Ferry,Rental Car Location,Sculpture Garden,Boutique,Coffee Shop,Airport Food Court
2,Central Bay Street,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Salad Place,Burger Joint,Department Store,Bubble Tea Shop,Japanese Restaurant,Pizza Place
3,Christie,Grocery Store,Café,Park,Candy Store,Diner,Italian Restaurant,Restaurant,Baby Store,Athletics & Sports,Nightclub
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Café,Pub,Men's Store,Mediterranean Restaurant,Hotel


## Examining K-Means Clustering

In [804]:
clusters = 5
toronto_grouped_clustering = res.drop('Neighbourhood', 1)
kmeans = KMeans(n_clusters=clusters, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_[0:10] 

array([0, 3, 0, 2, 0, 0, 0, 0, 0, 0], dtype=int32)

In [805]:

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = dffdt

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head()

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
84,Downtown Toronto,"Kensington Market, Chinatown, Grange Park",43.653206,-79.400049,0,Café,Coffee Shop,Vegetarian / Vegan Restaurant,Mexican Restaurant,Vietnamese Restaurant,Bar,Dessert Shop,Park,Pizza Place,Burger Joint
99,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Café,Pub,Men's Store,Mediterranean Restaurant,Hotel
80,Downtown Toronto,"University of Toronto, Harbord",43.662696,-79.400049,0,Café,Restaurant,Bookstore,Bar,Japanese Restaurant,Bakery,Sandwich Place,Italian Restaurant,Beer Bar,Beer Store
87,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420,3,Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Boat or Ferry,Rental Car Location,Sculpture Garden,Boutique,Coffee Shop,Airport Food Court
91,Downtown Toronto,Rosedale,43.679563,-79.377529,1,Park,Playground,Trail,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


In [806]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=14)
x = np.arange(clusters)
ys = [i + x + (i*x)**2 for i in range(clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examining Clusters

In [807]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
84,"Kensington Market, Chinatown, Grange Park",Café,Coffee Shop,Vegetarian / Vegan Restaurant,Mexican Restaurant,Vietnamese Restaurant,Bar,Dessert Shop,Park,Pizza Place,Burger Joint
99,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Café,Pub,Men's Store,Mediterranean Restaurant,Hotel
80,"University of Toronto, Harbord",Café,Restaurant,Bookstore,Bar,Japanese Restaurant,Bakery,Sandwich Place,Italian Restaurant,Beer Bar,Beer Store
92,Stn A PO Boxes,Coffee Shop,Italian Restaurant,Café,Restaurant,Seafood Restaurant,Beer Bar,Gym,Japanese Restaurant,Hotel,Cheese Shop
96,"St. James Town, Cabbagetown",Coffee Shop,Café,Restaurant,Pub,Pizza Place,Italian Restaurant,Bakery,Chinese Restaurant,Butcher,Playground
97,"First Canadian Place, Underground city",Coffee Shop,Café,Hotel,Restaurant,Gym,Japanese Restaurant,Deli / Bodega,Seafood Restaurant,Steakhouse,American Restaurant
2,"Regent Park, Harbourfront",Coffee Shop,Pub,Park,Bakery,Breakfast Spot,Café,Theater,Restaurant,Cosmetics Shop,Spa
48,"Commerce Court, Victoria Hotel",Coffee Shop,Café,Restaurant,Hotel,Gym,American Restaurant,Italian Restaurant,Seafood Restaurant,Japanese Restaurant,Deli / Bodega
9,"Garden District, Ryerson",Coffee Shop,Clothing Store,Café,Italian Restaurant,Japanese Restaurant,Bubble Tea Shop,Cosmetics Shop,Hotel,Ramen Restaurant,Middle Eastern Restaurant
42,"Toronto Dominion Centre, Design Exchange",Coffee Shop,Hotel,Café,Restaurant,Salad Place,Japanese Restaurant,Seafood Restaurant,Italian Restaurant,American Restaurant,Tea Room


In [808]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,Rosedale,Park,Playground,Trail,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


In [809]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,Christie,Grocery Store,Café,Park,Candy Store,Diner,Italian Restaurant,Restaurant,Baby Store,Athletics & Sports,Nightclub


In [810]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
87,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Boat or Ferry,Rental Car Location,Sculpture Garden,Boutique,Coffee Shop,Airport Food Court


In [811]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,"Queen's Park, Ontario Provincial Government",Coffee Shop,Diner,Yoga Studio,College Auditorium,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,Creperie
